<h2>[ Preview ]</h2>

<h4>
<ul>
    <li>Gridsearch</li>
    <li>Hyper parameter Tuning</li>
    <li>K-fold validation vs GridsearchCV</li>
    <li>Use GridSearchCV to find the best value of each parameter</li>
</ul>
</h4>

<h2>[ GridSearch ]</h2>

<h4>
<ul>
    <li>2-3. 코드 확인</li>
    <li>K-fold Validation의 복잡성 해결</li>
    <li>결과는 복잡하지만 DF로 표현하여 간략하게 확인 가능</li>
</ul>
</h4>

---

<h4>1. 데이터 처리</h4>

In [1]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df.head()


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [2]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [7]:
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df[45:55]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
45,4.8,3.0,1.4,0.3,setosa
46,5.1,3.8,1.6,0.2,setosa
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
53,5.5,2.3,4.0,1.3,versicolor
54,6.5,2.8,4.6,1.5,versicolor


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(['flower'], axis='columns'), df.flower, test_size=0.3)

In [16]:
from sklearn.svm import SVC

model = SVC(C=30, kernel="rbf", gamma="scale")
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9777777777777777

---

<h4>2. Hyper parameter Tuning</h4>

<h4>2-1. cross_val_score로 score 확인하기</h4>

In [21]:
from sklearn.model_selection import cross_val_score

cross_val_score(SVC(C=30, kernel="linear", gamma="scale"), iris.data, iris.target, cv=5)    


array([1. , 1. , 0.9, 0.9, 1. ])

In [19]:
cross_val_score(SVC(C=10, kernel='rbf', gamma='auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [20]:
cross_val_score(SVC(C=20, kernel='rbf', gamma='auto'), iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

<h4>2-2. cross_val_score로 다양한 파라미터의 정확도 확인</h4>

<ul>
    <li>parameter가 많은 경우 for문이 중첩으로 되어 루프가 많아짐</li>
    <li>시간이 오래걸리는 단점이 있음</li>
    <li>Gridsearch라는 API를 통해 동일 코드를 간단하게 수행</li>
</ul>

In [22]:
import numpy as np

kernels = ['rbf', 'linear', 'poly']
C = [1, 10, 20, 30, 100]
avg_scores={}

for _k in kernels:
    for _C in C:
        cv_scores = cross_val_score(SVC(C=_C, kernel=_k, gamma='auto'), iris.data, iris.target, cv=5)
        avg_scores[_k + '_' + str(_C)] = np.average(cv_scores)
        
avg_scores


{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'rbf_30': 0.96,
 'rbf_100': 0.96,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666,
 'linear_30': 0.96,
 'linear_100': 0.9666666666666666,
 'poly_1': 0.9666666666666666,
 'poly_10': 0.9666666666666666,
 'poly_20': 0.9533333333333334,
 'poly_30': 0.9533333333333334,
 'poly_100': 0.9466666666666667}

In [29]:
# parameter가 1개 증가하여 매우 복잡해짐

import numpy as np

kernel = ['rbf', 'linear', 'poly']
C = [1, 10, 20, 50, 100]
cv = [2, 5, 10]
avg_scores={}

for _k in kernel:
    for _C in C:
        for _cv in cv:
            scores = cross_val_score(SVC(kernel=_k, C=_C, gamma="auto"), iris.data, iris.target, cv=_cv)            
            avg_scores[_k + '_' + str(_C) + '_', str(_cv)] = np.average(scores)
            
avg_scores
                                     

{('rbf_1_', '2'): 0.9466666666666667,
 ('rbf_1_', '5'): 0.9800000000000001,
 ('rbf_1_', '10'): 0.9800000000000001,
 ('rbf_10_', '2'): 0.9733333333333334,
 ('rbf_10_', '5'): 0.9800000000000001,
 ('rbf_10_', '10'): 0.9733333333333334,
 ('rbf_20_', '2'): 0.94,
 ('rbf_20_', '5'): 0.9666666666666668,
 ('rbf_20_', '10'): 0.9666666666666668,
 ('rbf_50_', '2'): 0.9533333333333334,
 ('rbf_50_', '5'): 0.96,
 ('rbf_50_', '10'): 0.9600000000000002,
 ('rbf_100_', '2'): 0.9533333333333334,
 ('rbf_100_', '5'): 0.96,
 ('rbf_100_', '10'): 0.9666666666666668,
 ('linear_1_', '2'): 0.9666666666666667,
 ('linear_1_', '5'): 0.9800000000000001,
 ('linear_1_', '10'): 0.9733333333333334,
 ('linear_10_', '2'): 0.98,
 ('linear_10_', '5'): 0.9733333333333334,
 ('linear_10_', '10'): 0.9800000000000001,
 ('linear_20_', '2'): 0.9666666666666667,
 ('linear_20_', '5'): 0.9666666666666666,
 ('linear_20_', '10'): 0.9666666666666668,
 ('linear_50_', '2'): 0.96,
 ('linear_50_', '5'): 0.9666666666666666,
 ('linear_50_', '1

<h4>2-3. GridSearchCV</h4>

<ul>
    <li>GridSearchCV로 위의 코드를 간략하게 표현 가능</li>
    <li>결과는 보기 어렵게 나오지만 DF로 만들면 편하게 볼 수 있음</li>
    <li>clf.best_params으로 최적의 파라미터 확인</li>
</ul>

In [30]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(SVC(gamma='auto'),
                   {
                       'C' : [1,10,20],
                       'kernel' : ['rbf', 'linear']
                   },
                   cv=5, return_train_score=False)
clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00119848, 0.00059819, 0.00059853, 0.00059652, 0.00119681,
        0.00059824]),
 'std_fit_time': array([0.00039783, 0.00048842, 0.00048869, 0.00048707, 0.00039854,
        0.00048846]),
 'mean_score_time': array([0.00039873, 0.00039954, 0.00059762, 0.00040092, 0.00039845,
        0.00039897]),
 'std_score_time': array([0.00048834, 0.00048934, 0.00048796, 0.00049104, 0.00048799,
        0.00048864]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [31]:
_df = pd.DataFrame(clf.cv_results_)
_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001198,0.000398,0.000399,0.000488,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000598,0.000488,0.000400,0.000489,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000599,0.000489,0.000598,0.000488,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000597,0.000487,0.000401,0.000491,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.001197,0.000399,0.000398,0.000488,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000598,0.000488,0.000399,0.000489,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [32]:
_df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [39]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'multimetric_',
 'n_features_

In [35]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

<h4>2-4. RandomizedSearchCV</h4>

<ul>
    <li>계산 능력이 부족한 경우 임의로 파라미터를 선정하여 결과를 보여줌</li>
    <li>정확하지는 않지만 파라미터가 많은 경우 임의 시도</li>
</ul>

In [42]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rs.fit(iris.data, iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,20,rbf,0.966667
1,1,linear,0.980000


---

<h4>3. Best Model Selection with Hyper Parameter Tuning</h4>

<ul>
    <li>Model : SVM, RandomForestClassifier, LogisticRegression</li>
    <li>Use the GridSearchCV to find the best value of each parameter</li>
    <li>Check the best parameters in each model</li>
</ul>

In [52]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model' : SVC(gamma='auto'),
        'params' : {
            'C' : [1, 10, 20],
            'kernel' : ['rbf', 'linear', 'poly']
            }
    },
    'random_forest' : {
        'model' : RandomForestClassifier(),
        'params' : {
            'n_estimators' : [1, 5, 10]
        }
    },
    'logistic_regression' : {
        'model' :LogisticRegression(solver='liblinear',multi_class='auto'),
        'params' : {
            'C' : [1, 5, 10]
        }
    }
}

In [53]:
model_params.items()

dict_items([('svm', {'model': SVC(gamma='auto'), 'params': {'C': [1, 10, 20], 'kernel': ['rbf', 'linear', 'poly']}}), ('random_forest', {'model': RandomForestClassifier(), 'params': {'n_estimators': [1, 5, 10]}}), ('logistic_regression', {'model': LogisticRegression(solver='liblinear'), 'params': {'C': [1, 5, 10]}})])

In [57]:
scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model' : model_name,
        'best_score' : clf.best_score_,
        'best_params' : clf.best_params_
    })
df = pd.DataFrame(scores, columns = ['model', 'best_score', 'best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}


In [58]:
scores

[{'model': 'svm',
  'best_score': 0.9800000000000001,
  'best_params': {'C': 1, 'kernel': 'rbf'}},
 {'model': 'random_forest',
  'best_score': 0.9600000000000002,
  'best_params': {'n_estimators': 5}},
 {'model': 'logistic_regression',
  'best_score': 0.9666666666666668,
  'best_params': {'C': 5}}]